# Getting reservoirs from the GWW project

either by hand:

In [ ]:
import json
import os
from pathlib import Path
import subprocess

output: subprocess.CompletedProcess = subprocess.run(["mamba", "list", "^proj", "--json"], capture_output=True)
proj_dist_name: str = json.loads(output.stdout)[0]["dist_name"]
os.environ["PROJ_LIB"]=f"/opt/conda/pkgs/proj-{proj_dist_name}"

In [ ]:
import fiona
from google.cloud.storage import Blob, Bucket, Client

In [ ]:
gcp_project: str = "global-water-watch"
gcp_bucket: str = "global-water-watch"

os.environ["GCLOUD_PROJECT"] = "global-water-watch"

out_dir: Path = Path.cwd() / "output"

In [ ]:
client: Client = Client()
bucket: Bucket = Bucket(client, name=gcp_bucket, user_project=gcp_project)

In [ ]:
reservoir_dir: Path = out_dir / "reservoirs"
blob: Blob = client.list_blobs(bucket, prefix="shp/reservoirs-v1.0")
for b in blob:
    b.download_to_filename(reservoir_dir / b.name.split("/")[-1])

In [ ]:
shapes = fiona.open(reservoir_dir / "reservoirs-v1.0.shp")

In [ ]:
next(iter(shapes))

In [ ]:
from functools import reduce

def process_shapes(collection: dict, shape: dict):
    fid: int = int(shape["properties"]["fid"])
    s_type: str = shape["geometry"]["type"]
    coords: List = shape["geometry"]["coordinates"]
    if s_type == "MultiPolygon":
        collection.update({fid: MultiPolygon([Polygon(c[0]) for c in coords])})
    elif s_type == "Polygon":
        collection.update({fid: Polygon(coords[0])})
    return collection

p_shapes = reduce(process_shapes, iter(shapes), {})

In [ ]:
p_shapes[60000]

# With class:

In [ ]:
from utils import Reservoir
from pathlib import Path

reservoir_dir: Path = Path.cwd() / "output" / "reservoirs"
reservoirs = Reservoir.from_gcp(reservoir_dir)

In [ ]:
import geojson
import shapely

geojson.MultiPolygon(shapely.geometry.mapping(next(filter(lambda r: r.fid() == 1, reservoirs)).geometry)["coordinates"])["coordinates"]

In [ ]:
next(filter(lambda r: r.fid() == 1, reservoirs)).geometry.wkt

# Filter all reservoirs within bbox

For the workflow in mind, we want to filter all reservoirs within the bounding box of the area selected.
Getting the reservoir in question: https://code.earthengine.google.com/bf5e67d0efc01b434ddc23c0db74e642

In [ ]:
import geojson
from shapely.geometry import MultiPolygon, Polygon

shapely_polygon_chzech = Polygon([[12.09,51.06],[12.09, 48.55], [18.87,48.55], [18.87, 51.06], [12.09,51.06]])
geojson_str = "{\"type\":\"Polygon\",\"coordinates\":[[[16.258372886421807,49.561646293673824],[16.314909857006697,49.561646293673824],[16.314909857006697,49.58980547068479],[16.258372886421807,49.58980547068479],[16.258372886421807,49.561646293673824]]],\"geodesic\":false}"
bbox: geojson.Polygon = geojson.loads(geojson_str)
shapely_polygon = Polygon(bbox.coordinates[0]).buffer(0.001)  # need to slightly buffer to contain reservoir

selected = list(filter(lambda r: shapely_polygon.covers(r.geometry), reservoirs))

In [ ]:
len(selected)

In [ ]:
selected[0].geometry